In [1]:
import pandas as pd

In [2]:
cd ../

/home/anna/PycharmProjects/DataJournalismApp


In [10]:
import util.data_preprocessing as dp
import util.keywords_extraction as ke
import util.topic_extraction as te

In [4]:
data = pd.read_csv("data/raw_data/news_v2.csv", index_col=0)
data.head()

,date,rubric,tags,title,descr,text
index,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура Владимирской области,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...","Прокуратура обнаружила, что паспорт региональн..."
1,2019-08-17,Новости 33; Происшествия 33,Гусь-Хрустальный мошенники,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,Следственный отдел полиции в Гусь-Хрустальном ...
2,2019-08-16,Новости 33; Общество,Жириновский во Владимире 16.08.2019,"Жириновский: ""Мое имя тоже - Владимир. Всё сош...",Во Владимирском академическом театре драмы про...,Во Владимирском академическом театре драмы про...
3,2019-08-16,Новости 33; Общество,"модернизация тепловых сетей Владимир, Т Плюс",Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,Во Владимире модернизируют тепловую сеть на ул...
4,2019-08-16,Новости 33; Общество,"ВлГУ, Евгений Седов, СпбГУ",Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,Выпускник кафедры физики и прикладной математи...


In [6]:
data = dp.preprocess_data(data.drop(["tags", "descr"], axis=1), ["title", "text"], "news33", time_col="date")
data.head()

,date,rubric,title,text,_time,source
index,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33


In [14]:
data = data[:100]

In [15]:
corpus = data["text"]
all_text_keywords, vectorizer = ke.get_keywords(corpus)

In [16]:
data["keywords"] = all_text_keywords
data.head()

,date,rubric,title,text,_time,source,keywords
index,,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33,фапах; фап; медицинский; пункт; район; деятель...
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33,женщина; хрустальный; гусь; потерпевший; звони...
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33,жириновский; губернатор; сипягин; собрание; лд...
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33,теплов; составить; сетей; отопительный; м²; ра...
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33,скорость; физика; света; поток; полупроводнико...


In [18]:
ke.get_keywords_preprocessed(data["text"][:10], vectorizer)

['фапах; фап; медицинский; пункт; район; деятельность; района; фапов; пункта; прокуратура',
 'женщина; хрустальный; гусь; потерпевший; звонить; адвокаты; компенсация; звонок; неизвестный; кредит',
 'жириновский; губернатор; сипягин; собрание; лдпр; владимир; областной; обещать; лидер; регион',
 'теплов; составить; сетей; отопительный; м²; работа; подготовка; возле; вода; филиал',
 'скорость; физика; света; поток; полупроводниковый; квантовый; вычислительный; создание; самый; информация',
 'брат; старший; приговорить; строгий; признать; произойти; хотел; угар; телесный; судогде',
 'совет; территорий; генплан; архитектор; проект; застройка; областной; культурный; объект; города',
 'жириновский; региона; губернатор; владимир; торжество; телеграмма; сипягиным; предполагаться; правительственный; почтовый',
 'проверка; дорожный; хрустальный; гусь; автомобильный; прокурорский; мошенничество; выявить; контракт; деревня',
 'комбинат; тепличный; ленинский; крупный; виктор; приговор; размере; дох

In [20]:
topic_keywords, topics_pred, vectorizer, lda = te.get_topics(data["text"].values, topics=5, n_keywords=5)

In [21]:
topic_keywords

{0: ['владимирский', 'область', 'суд', 'владимир', 'дом'],
 1: ['владимирский', 'владимир', 'житель', 'администрация', 'область'],
 2: ['август', 'служба', 'владимирский', 'житель', 'дтп'],
 3: ['свой', 'владимирский', 'убийство', 'суд', 'летний'],
 4: ['владимирский', 'август', 'владимир', 'свой', 'рублей']}

In [24]:
topics = {0: "суд", 1: "социум", 2: "дтп", 3: "убийство", 4: "август"}

In [23]:
topics_pred

array([4, 2, 1, 1, 3, 3, 4, 1, 0, 2, 2, 1, 1, 3, 2, 2, 4, 2, 1, 2, 2, 0,
       4, 1, 3, 2, 1, 4, 4, 4, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 4,
       3, 1, 4, 1, 0, 3, 1, 4, 4, 0, 0, 4, 0, 1, 1, 0, 1, 1, 2, 0, 1, 0,
       0, 1, 4, 4, 4, 2, 1, 0, 0, 4, 1, 0, 0, 1, 2, 4, 4, 1, 4, 1, 4, 3,
       2, 4, 1, 1, 0, 4, 4, 1, 1, 3, 4, 1])

In [26]:
te.get_topics_pretrained(data["text"].values[:10], lda, vectorizer)

array([4, 2, 1, 1, 3, 3, 4, 1, 0, 2])

In [29]:
data = te.set_topics_to_data(data, topics_pred, topics)
data.head()

,date,rubric,title,text,_time,source,keywords,topic
index,,,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33,фапах; фап; медицинский; пункт; район; деятель...,август
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33,женщина; хрустальный; гусь; потерпевший; звони...,дтп
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33,жириновский; губернатор; сипягин; собрание; лд...,социум
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33,теплов; составить; сетей; отопительный; м²; ра...,социум
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33,скорость; физика; света; поток; полупроводнико...,убийство
